In [1]:
#%matplotlib inline 
import torch 
#from IPython import display 
#from matplotlib import pyplot as plt
import numpy as np 
import random
import math
from time import time
#from torch import optim

In [2]:
def generatedata(noise,showpermutation=False,showtrue_w=False):
    true_w2 = torch.from_numpy(np.random.normal(0, 1,(num_X2feature,1)))
    if showtrue_w:
        print('true_w2:',true_w2)
    X2_before_ =torch.from_numpy(np.random.normal(0, 1, (num_example, num_X2feature)))
    y_ = torch.mm(X2_before_,true_w2)
    y_ += torch.from_numpy(np.random.normal(0, noise ,size=y_.size()))
    P_array=np.random.permutation(num_example)
    P=torch.zeros(num_example,num_example,dtype=torch.float64)
    for i in range(num_example):
        P[i][P_array[i]]=1
    if showpermutation:
        print('打乱X2的置换矩阵为',P)
    X2_=torch.mm(P,X2_before_)
    #X2_=X2_before_
    error_reg=(torch.norm(y_-torch.mm(X2_before_,true_w2))/torch.norm(y_))
    return y_,X2_,true_w2,P,error_reg

In [3]:
def generateinitialw(method,showinitialw=False):
    if method=='normal':
        w2 = torch.from_numpy(np.random.normal(0, 1,(num_X2feature,1)))
    if method=='zeros':
        w2=torch.zeros(num_X2feature,1,dtype=torch.float64)
    if showinitialw:
        print('initial w2:',w2)
    return w2

In [4]:
def sinkhorn_stabilized(a, b, M, reg, numItermax=1000, tau=1e3, stopThr=1e-9,
                        warmstart=None, verbose=False, print_period=20,
                        log=False, **kwargs):

#     a = np.asarray(a, dtype=np.float64)
#     b = np.asarray(b, dtype=np.float64)
#     M = np.asarray(M, dtype=np.float64)
    a=a
    b=b
    M=M

#     if len(a) == 0:
#         a = np.ones((M.shape[0],), dtype=np.float64) / M.shape[0]
#     if len(b) == 0:
#         b = np.ones((M.shape[1],), dtype=np.float64) / M.shape[1]

    # test if multiple target
#     if len(b.shape) > 1:
#         n_hists = b.shape[1]
#         a = a[:, np.newaxis]
#     else:
#         n_hists = 0
    n_hists = 0
    # init data
    dim_a = len(a)
    dim_b = len(b)

    cpt = 0
    if log:
        log = {'err': []}

    # we assume that no distances are null except those of the diagonal of
    # distances
    if warmstart is None:
        alpha, beta = torch.zeros(dim_a,1,dtype=torch.float64), torch.zeros(dim_b,1,dtype=torch.float64)
    else:
        alpha, beta = warmstart

    if n_hists:
        u = torch.ones((dim_a, n_hists)) / dim_a
        v = torch.ones((dim_b, n_hists)) / dim_b
    else:
        u, v = torch.ones(dim_a,1,dtype=torch.float64) / dim_a, torch.ones(dim_b,1,dtype=torch.float64) / dim_b

    def get_K(alpha, beta):
        """log space computation"""
        return torch.exp(-(M - alpha.reshape((dim_a, 1))- beta.reshape((1, dim_b))) / reg)

    def get_Gamma(alpha, beta, u, v):
        """log space gamma computation"""
        return torch.exp(-(M - alpha.reshape((dim_a, 1)) - beta.reshape((1, dim_b)))
                      / reg + torch.log(u.reshape((dim_a, 1))) + torch.log(v.reshape((1, dim_b))))

    # print(np.min(K))

    K = get_K(alpha, beta)
    transp = K
    loop = 1
    cpt = 0
    err = 1
    while loop:

        uprev = u
        vprev = v
        # sinkhornrn update
        v = b / (torch.mm(K.transpose(1,0), u) + 1e-16)
        u = a / (torch.mm(K, v) + 1e-16)
        # remove numerical problems and store them in K
        if torch.abs(u).max() > tau or torch.abs(v).max() > tau:
            if n_hists:
                alpha, beta = alpha + reg * \
                    torch.max(torch.log(u), 1), beta + reg * torch.max(np.log(v))
            else:
                alpha, beta = alpha + reg * torch.log(u), beta + reg * torch.log(v)
                if n_hists:
                    u, v = torch.ones((dim_a, n_hists)) / dim_a, torch.ones((dim_b, n_hists)) / dim_b
                else:
                    u, v = torch.ones(dim_a,1,dtype=torch.float64) / dim_a, torch.ones(dim_b,1,dtype=torch.float64) / dim_b
            K = get_K(alpha, beta)
            

        if cpt % print_period == 0:
            # we can speed up the process by checking for the error only all
            # the 10th iterations
            if n_hists:
                err_u = abs(u - uprev).max()
                err_u /= max(abs(u).max(), abs(uprev).max(), 1.)
                err_v = abs(v - vprev).max()
                err_v /= max(abs(v).max(), abs(vprev).max(), 1.)
                err = 0.5 * (err_u + err_v)
            else:
                transp = get_Gamma(alpha, beta, u, v)
                err = torch.norm((torch.sum(transp, axis=0) - b))
            if log:
                log['err'].append(err)

            if verbose:
                if cpt % (print_period * 20) == 0:
                    print(
                        '{:5s}|{:12s}'.format('It.', 'Err') + '\n' + '-' * 19)
                print('{:5d}|{:8e}|'.format(cpt, err))

        if err <= stopThr:
            loop = False

        if cpt >= numItermax:
            loop = False

        if np.any(np.isnan(u.detach().numpy())) or np.any(np.isnan(v.detach().numpy())):
            # we have reached the machine precision
            # come back to previous solution and quit loop
            print('Warning: numerical errors at iteration', cpt)
            u = uprev
            v = vprev
            break

        cpt = cpt + 1
    #print(cpt)
    if log:
        if n_hists:
            alpha = alpha[:, None]
            beta = beta[:, None]
        logu = alpha / reg + torch.log(u)
        logv = beta / reg + torch.log(v)
        log['logu'] = logu
        log['logv'] = logv
        log['alpha'] = alpha + reg * torch.log(u)
        log['beta'] = beta + reg * torch.log(v)
        log['warmstart'] = (log['alpha'], log['beta'])
        if n_hists:
            res = torch.zeros((n_hists))
            for i in range(n_hists):
                res[i] = torch.sum(get_Gamma(alpha, beta, u[:, i], v[:, i]) * M)
            return res, log

        else:
            return get_Gamma(alpha, beta, u, v), log
    else:
        if n_hists:
            res = torch.zeros((n_hists))
            for i in range(n_hists):
                res[i] = torch.sum(get_Gamma(alpha, beta, u[:, i], v[:, i]) * M)
            return res
        else:
            return get_Gamma(alpha, beta, u, v)

In [5]:
def sinkhorn_epsilon_scaling(a, b, M, reg, numItermax=100, epsilon0=1e4,
                             numInnerItermax=100, tau=1e3, stopThr=1e-9,
                             warmstart=None, verbose=False, print_period=10,
                             log=False, **kwargs):
    #a = np.asarray(a, dtype=np.float64)
    #b = np.asarray(b, dtype=np.float64)
    #M = np.asarray(M, dtype=np.float64)
    a=a
    b=b
    M=M
#     if len(a) == 0:
#         a = np.ones((M.shape[0],), dtype=np.float64) / M.shape[0]
#     if len(b) == 0:
#         b = np.ones((M.shape[1],), dtype=np.float64) / M.shape[1]

    # init data
    dim_a = len(a)
    #dim_a=num_example
    dim_b = len(b)
    #dim_b=num_example
    # nrelative umerical precision with 64 bits
    numItermin = 35
    numItermax = max(numItermin, numItermax)  # ensure that last velue is exact

    cpt = 0
    if log:
        log = {'err': []}

    # we assume that no distances are null except those of the diagonal of
    # distances
    if warmstart is None:
        alpha, beta = torch.zeros(dim_a,1,dtype=torch.float64), torch.zeros(dim_b,1,dtype=torch.float64)
    else:
        alpha, beta = warmstart

    def get_K(alpha, beta):
        """log space computation"""
        return torch.exp(-(M - alpha.reshape((dim_a, 1))
                        - beta.reshape((1, dim_b))) / reg)

    # print(np.min(K))
    def get_reg(n):  # exponential decreasing
        return (epsilon0 - reg) * np.exp(-n) + reg

    loop = 1
    cpt = 0
    err = 1
    while loop:

        regi = get_reg(cpt)

        G, logi = sinkhorn_stabilized(a, b, M, regi,
                                      numItermax=numInnerItermax, stopThr=1e-9,
                                      warmstart=(alpha, beta), verbose=False,
                                      print_period=20, tau=tau, log=True)

        alpha = logi['alpha']
        beta = logi['beta']

        if cpt >= numItermax:
            loop = False

        if cpt % (print_period) == 0:  # spsion nearly converged
            # we can speed up the process by checking for the error only all
            # the 10th iterations
            transp = G
            err = torch.norm(
                (torch.sum(transp, axis=0) - b))**2 + torch.norm((torch.sum(transp, axis=1) - a))**2
            if log:
                log['err'].append(err)

            if verbose:
                if cpt % (print_period * 10) == 0:
                    print(
                        '{:5s}|{:12s}'.format('It.', 'Err') + '\n' + '-' * 19)
                print('{:5d}|{:8e}|'.format(cpt, err))

        if err <= stopThr and cpt > numItermin:
            loop = False

        cpt = cpt + 1
    # print('err=',err,' cpt=',cpt)
    if log:
        log['alpha'] = alpha
        log['beta'] = beta
        log['warmstart'] = (log['alpha'], log['beta'])
        return G, log
    else:
        return G
    

In [14]:
gama_=1
eta=0
starts=1
for i_____ in range(5):
    for num_example in range(150,151,100): 
        for num_X2feature in [30]:
            for i____ in range(10):
                (y_,X2_,true_w2,true_P,error_reg1)=generatedata(noise=0.3)
                y=y_
                X2=X2_
                results_Loss = []
                results_w2=[]
                results_error=[]
                for i__ in range(starts):
    #                     P_array=np.random.permutation(num_example)
    #                     P=torch.zeros(num_example,num_example,dtype=torch.float64)
    #                     for i in range(num_example):
    #                         P[i][P_array[i]]=1
    #                     X_=torch.cat([X1,X2],1)
    #                     X=torch.mm(P,X_)
    #                     w=torch.mm(torch.mm(torch.tensor(np.linalg.inv(torch.mm(X.transpose(1,0),X))),X.transpose(1,0)),y)
    #                     w1,w2=w.split([num_X1feature,num_X2feature],dim=0)
    #                     w1=torch.from_numpy(np.random.normal(0, 0,(num_X1feature,1)))
    #                     w2=torch.from_numpy(np.random.normal(0, 0,(num_X2feature,1)))
                    w2=generateinitialw(method='zeros')
                    #w2=true_w2
                    w2.requires_grad_(requires_grad=True)
    #                 results_Loss = []
                    lr=0.0025
                    results_S=[]
                    t=0
                    before1=0
                    while True:
                        #start=time()
                        Y1=y
                        Y2=torch.mm(X2,w2)
                        C=torch.zeros(num_example,num_example,dtype=torch.float64)
                        for i in range(num_example):
                            for j in range(num_example):
                                C[i][j]=(Y1[i]-Y2[j])**2            

                        #S=SinkhornIPOT(C)
                        a=torch.ones(num_example,1,dtype=torch.float64)
                        b=torch.ones(num_example,1,dtype=torch.float64)
                        S=sinkhorn_epsilon_scaling(a, b, C, 0.00000001)
                        #print(S.transpose(1,0).half())
                        #results_S.append(S)
                        #if t>0:
                            #print('        S变化',(torch.norm(results_S[t]-results_S[t-1]))/(torch.norm(results_S[t-1])))
                        #Loss=torch.sum(S*C)
                        Loss=torch.norm(Y1-torch.mm(S,Y2))**2
                        if Loss<1e-2:
                            break
                        Loss.backward()
    #                         results_Loss.append(Loss)
    #                         for i_ in range(num_X1features):
    #                             results_w1[t][i_]=(w1[i_].data)
    #                         for i_ in range(num_X2features):
    #                             results_w2[t][i_]=(w2[i_].data)
                        w2.data-=lr*(w2.grad+np.random.normal(0,np.sqrt(eta/(1+t)**gama_)))
                        #print(w2.grad)
    #                     if t==num_epochs-1:
    #                         print('最终w1梯度：',w1.grad)
    #                         print('最终w2梯度：',w2.grad)
                        w2.grad.data.zero_() 
                        print('Loss',t,'=',Loss.data)
    #                     if t%6==0:
    #                         if torch.norm(Loss-before1)<1e-4:
    #                             break
    #                         before1=Loss
                        if torch.norm(Loss-before1)/before1<1e-7:
                            break
                        before1=Loss
                        if t>=200:
                            print('超过迭代上限')
                            break
                        if math.isnan(Loss):
                            break
                        t+=1
                        print('#',end='')
                        #print(time()-start)



                    print(' ',end='')
                    error_each=(torch.norm(w2-true_w2))
                    results_error.append(error_each)
                    #results_Loss.append(Loss)
                    #results_w1.append(w1.data)
                    #results_w2.append(w2.data)


                #w1=results_w1[results_Loss.index(min(results_Loss))]
                #w2=results_w2[results_Loss.index(min(results_Loss))]

    #                     for i_ in range(starts):
    #                         results_w1[i_]=(w1[i_].data)
    #                     for i_ in range(starts):
    #                         results_w2[i_]=(w2[i_].data)


                #error_w=((torch.norm(w1-true_w1))/(torch.norm(true_w1))+(torch.norm(w2-true_w2))/(torch.norm(true_w2)))/2
                #print(num_X1feature,num_X2feature,num_example,'平均相对误差1：',error_w)
                print(num_X2feature,num_example,'平均相对误差2：',np.min(results_error),end='   ')

                #print('真实置换矩阵为：',true_P)
                error_P=(torch.norm(S.transpose(1,0)-true_P))/(torch.norm(true_P))
                print(num_X2feature,num_example,'置换矩阵误差：',error_P,end='   ')
                error_reg2=(torch.norm(y_-torch.mm(torch.mm(S,X2_),w2))/torch.norm(y_))
                print('真实回归误差',error_reg1,end='   ')
                print('实验回归误差',error_reg2)
                #print('双随机矩阵S为：',S.transpose(1,0).half())
                #print(results)
    #                 plt.figure(figsize=(6,6))
    #                 plt.plot(results_w1_0,results_Loss, '-o',label='$w1[0]$')
    #                 plt.plot(results_w1_1,results_Loss, '-o',label='$w1[1]$')
    #                 plt.plot(results_w1_2,results_Loss, '-o',label='$w1[2]$')
    #                 plt.plot(results_w2_0,results_Loss, '-o',label='$w2[0]$')
    #                 plt.plot(results_w2_1,results_Loss, '-o',label='$w2[1]$')
    #                 plt.plot(results_w2_2,results_Loss, '-o',label='$w2[2]$')
    #                 plt.legend()
    #                 plt.show()

Loss 0 = tensor(3119.0879, dtype=torch.float64)
#Loss 1 = tensor(3058.8860, dtype=torch.float64)
#Loss 2 = tensor(214.5526, dtype=torch.float64)
#Loss 3 = tensor(193.1970, dtype=torch.float64)
#Loss 4 = tensor(179.5074, dtype=torch.float64)
#Loss 5 = tensor(168.4188, dtype=torch.float64)
#Loss 6 = tensor(160.0754, dtype=torch.float64)
#Loss 7 = tensor(152.1518, dtype=torch.float64)
#Loss 8 = tensor(144.4730, dtype=torch.float64)
#Loss 9 = tensor(136.7875, dtype=torch.float64)
#Loss 10 = tensor(129.4504, dtype=torch.float64)
#Loss 11 = tensor(124.1337, dtype=torch.float64)
#Loss 12 = tensor(119.8356, dtype=torch.float64)
#Loss 13 = tensor(114.4960, dtype=torch.float64)
#Loss 14 = tensor(109.2088, dtype=torch.float64)
#Loss 15 = tensor(104.5247, dtype=torch.float64)
#Loss 16 = tensor(99.2911, dtype=torch.float64)
#Loss 17 = tensor(94.2866, dtype=torch.float64)
#Loss 18 = tensor(89.5660, dtype=torch.float64)
#Loss 19 = tensor(84.8250, dtype=torch.float64)
#Loss 20 = tensor(78.5670, dtype=

#Loss 3 = tensor(277.4092, dtype=torch.float64)
#Loss 4 = tensor(231.9813, dtype=torch.float64)
#Loss 5 = tensor(199.0891, dtype=torch.float64)
#Loss 6 = tensor(174.4242, dtype=torch.float64)
#Loss 7 = tensor(151.3327, dtype=torch.float64)
#Loss 8 = tensor(136.2763, dtype=torch.float64)
#Loss 9 = tensor(122.2989, dtype=torch.float64)
#Loss 10 = tensor(110.0395, dtype=torch.float64)
#Loss 11 = tensor(101.0484, dtype=torch.float64)
#Loss 12 = tensor(92.4008, dtype=torch.float64)
#Loss 13 = tensor(85.1763, dtype=torch.float64)
#Loss 14 = tensor(77.3370, dtype=torch.float64)
#Loss 15 = tensor(70.6308, dtype=torch.float64)
#Loss 16 = tensor(65.4989, dtype=torch.float64)
#Loss 17 = tensor(61.2651, dtype=torch.float64)
#Loss 18 = tensor(57.8708, dtype=torch.float64)
#Loss 19 = tensor(55.1595, dtype=torch.float64)
#Loss 20 = tensor(52.2021, dtype=torch.float64)
#Loss 21 = tensor(49.2212, dtype=torch.float64)
#Loss 22 = tensor(46.6838, dtype=torch.float64)
#Loss 23 = tensor(43.9872, dtype=torch

#Loss 50 = tensor(12.5141, dtype=torch.float64)
#Loss 51 = tensor(12.0972, dtype=torch.float64)
#Loss 52 = tensor(11.6705, dtype=torch.float64)
#Loss 53 = tensor(11.1726, dtype=torch.float64)
#Loss 54 = tensor(10.7840, dtype=torch.float64)
#Loss 55 = tensor(10.3023, dtype=torch.float64)
#Loss 56 = tensor(9.7812, dtype=torch.float64)
#Loss 57 = tensor(9.3522, dtype=torch.float64)
#Loss 58 = tensor(8.9876, dtype=torch.float64)
#Loss 59 = tensor(8.6495, dtype=torch.float64)
#Loss 60 = tensor(8.3083, dtype=torch.float64)
#Loss 61 = tensor(8.1315, dtype=torch.float64)
#Loss 62 = tensor(7.9189, dtype=torch.float64)
#Loss 63 = tensor(7.7102, dtype=torch.float64)
#Loss 64 = tensor(7.4142, dtype=torch.float64)
#Loss 65 = tensor(7.2235, dtype=torch.float64)
#Loss 66 = tensor(7.0698, dtype=torch.float64)
#Loss 67 = tensor(6.9655, dtype=torch.float64)
#Loss 68 = tensor(6.8120, dtype=torch.float64)
#Loss 69 = tensor(6.6858, dtype=torch.float64)
#Loss 70 = tensor(6.5986, dtype=torch.float64)
#Loss 7

#Loss 39 = tensor(6.6974, dtype=torch.float64)
#Loss 40 = tensor(6.4198, dtype=torch.float64)
#Loss 41 = tensor(6.3123, dtype=torch.float64)
#Loss 42 = tensor(6.1525, dtype=torch.float64)
#Loss 43 = tensor(6.0564, dtype=torch.float64)
#Loss 44 = tensor(5.9986, dtype=torch.float64)
#Loss 45 = tensor(5.8787, dtype=torch.float64)
#Loss 46 = tensor(5.7680, dtype=torch.float64)
#Loss 47 = tensor(5.6996, dtype=torch.float64)
#Loss 48 = tensor(5.6152, dtype=torch.float64)
#Loss 49 = tensor(5.5335, dtype=torch.float64)
#Loss 50 = tensor(5.4660, dtype=torch.float64)
#Loss 51 = tensor(5.3594, dtype=torch.float64)
#Loss 52 = tensor(5.3238, dtype=torch.float64)
#Loss 53 = tensor(5.2515, dtype=torch.float64)
#Loss 54 = tensor(5.1442, dtype=torch.float64)
#Loss 55 = tensor(5.0416, dtype=torch.float64)
#Loss 56 = tensor(5.0040, dtype=torch.float64)
#Loss 57 = tensor(4.9585, dtype=torch.float64)
#Loss 58 = tensor(4.9464, dtype=torch.float64)
#Loss 59 = tensor(4.9299, dtype=torch.float64)
#Loss 60 = te

#Loss 43 = tensor(13.2578, dtype=torch.float64)
#Loss 44 = tensor(12.4165, dtype=torch.float64)
#Loss 45 = tensor(11.7716, dtype=torch.float64)
#Loss 46 = tensor(11.0806, dtype=torch.float64)
#Loss 47 = tensor(10.6226, dtype=torch.float64)
#Loss 48 = tensor(10.3237, dtype=torch.float64)
#Loss 49 = tensor(10.0721, dtype=torch.float64)
#Loss 50 = tensor(9.7535, dtype=torch.float64)
#Loss 51 = tensor(9.5995, dtype=torch.float64)
#Loss 52 = tensor(9.3307, dtype=torch.float64)
#Loss 53 = tensor(8.8136, dtype=torch.float64)
#Loss 54 = tensor(8.5081, dtype=torch.float64)
#Loss 55 = tensor(8.2456, dtype=torch.float64)
#Loss 56 = tensor(8.0053, dtype=torch.float64)
#Loss 57 = tensor(7.7541, dtype=torch.float64)
#Loss 58 = tensor(7.5751, dtype=torch.float64)
#Loss 59 = tensor(7.3546, dtype=torch.float64)
#Loss 60 = tensor(7.1207, dtype=torch.float64)
#Loss 61 = tensor(7.0207, dtype=torch.float64)
#Loss 62 = tensor(6.9022, dtype=torch.float64)
#Loss 63 = tensor(6.8367, dtype=torch.float64)
#Loss 

...

#Loss 2 = tensor(558.3925, dtype=torch.float64)
#Loss 3 = tensor(460.9618, dtype=torch.float64)
#Loss 4 = tensor(391.1160, dtype=torch.float64)
#Loss 5 = tensor(321.9297, dtype=torch.float64)
#Loss 6 = tensor(260.2871, dtype=torch.float64)
#Loss 7 = tensor(210.6620, dtype=torch.float64)
#Loss 8 = tensor(170.5316, dtype=torch.float64)
#Loss 9 = tensor(139.7664, dtype=torch.float64)
#Loss 10 = tensor(115.6211, dtype=torch.float64)
#Loss 11 = tensor(98.2049, dtype=torch.float64)
#Loss 12 = tensor(86.1559, dtype=torch.float64)
#Loss 13 = tensor(74.6476, dtype=torch.float64)
#Loss 14 = tensor(64.1036, dtype=torch.float64)
#Loss 15 = tensor(56.5076, dtype=torch.float64)
#Loss 16 = tensor(50.8314, dtype=torch.float64)
#Loss 17 = tensor(44.8986, dtype=torch.float64)
#Loss 18 = tensor(39.3652, dtype=torch.float64)
#Loss 19 = tensor(35.4789, dtype=torch.float64)
#Loss 20 = tensor(31.3335, dtype=torch.float64)
#Loss 21 = tensor(27.8747, dtype=torch.float64)
#Loss 22 = tensor(25.1295, dtype=torch.

Loss 0 = tensor(5565.2014, dtype=torch.float64)
#Loss 1 = tensor(5448.7715, dtype=torch.float64)
#Loss 2 = tensor(595.9290, dtype=torch.float64)
#Loss 3 = tensor(514.6246, dtype=torch.float64)
#Loss 4 = tensor(439.1465, dtype=torch.float64)
#Loss 5 = tensor(371.3500, dtype=torch.float64)
#Loss 6 = tensor(319.0733, dtype=torch.float64)
#Loss 7 = tensor(270.3551, dtype=torch.float64)
#Loss 8 = tensor(232.6454, dtype=torch.float64)
#Loss 9 = tensor(192.0392, dtype=torch.float64)
#Loss 10 = tensor(158.2352, dtype=torch.float64)
#Loss 11 = tensor(131.4624, dtype=torch.float64)
#Loss 12 = tensor(107.9064, dtype=torch.float64)
#Loss 13 = tensor(89.1072, dtype=torch.float64)
#Loss 14 = tensor(74.1116, dtype=torch.float64)
#Loss 15 = tensor(62.0867, dtype=torch.float64)
#Loss 16 = tensor(53.1918, dtype=torch.float64)
#Loss 17 = tensor(44.5462, dtype=torch.float64)
#Loss 18 = tensor(37.5360, dtype=torch.float64)
#Loss 19 = tensor(31.7049, dtype=torch.float64)
#Loss 20 = tensor(27.1002, dtype=tor

#Loss 13 = tensor(140.9746, dtype=torch.float64)
#Loss 14 = tensor(130.5768, dtype=torch.float64)
#Loss 15 = tensor(117.6145, dtype=torch.float64)
#Loss 16 = tensor(104.9357, dtype=torch.float64)
#Loss 17 = tensor(94.1492, dtype=torch.float64)
#Loss 18 = tensor(84.1461, dtype=torch.float64)
#Loss 19 = tensor(76.2109, dtype=torch.float64)
#Loss 20 = tensor(69.3863, dtype=torch.float64)
#Loss 21 = tensor(63.6514, dtype=torch.float64)
#Loss 22 = tensor(59.2621, dtype=torch.float64)
#Loss 23 = tensor(54.3573, dtype=torch.float64)
#Loss 24 = tensor(50.3811, dtype=torch.float64)
#Loss 25 = tensor(47.2249, dtype=torch.float64)
#Loss 26 = tensor(43.2716, dtype=torch.float64)
#Loss 27 = tensor(39.9589, dtype=torch.float64)
#Loss 28 = tensor(37.7335, dtype=torch.float64)
#Loss 29 = tensor(36.3879, dtype=torch.float64)
#Loss 30 = tensor(35.3143, dtype=torch.float64)
#Loss 31 = tensor(34.5531, dtype=torch.float64)
#Loss 32 = tensor(33.6690, dtype=torch.float64)
#Loss 33 = tensor(32.8275, dtype=tor

#Loss 11 = tensor(118.2864, dtype=torch.float64)
#Loss 12 = tensor(104.6817, dtype=torch.float64)
#Loss 13 = tensor(90.3347, dtype=torch.float64)
#Loss 14 = tensor(80.2921, dtype=torch.float64)
#Loss 15 = tensor(72.5967, dtype=torch.float64)
#Loss 16 = tensor(63.7610, dtype=torch.float64)
#Loss 17 = tensor(57.1346, dtype=torch.float64)
#Loss 18 = tensor(52.9285, dtype=torch.float64)
#Loss 19 = tensor(49.0968, dtype=torch.float64)
#Loss 20 = tensor(45.0905, dtype=torch.float64)
#Loss 21 = tensor(41.6071, dtype=torch.float64)
#Loss 22 = tensor(38.9161, dtype=torch.float64)
#Loss 23 = tensor(36.4066, dtype=torch.float64)
#Loss 24 = tensor(34.9034, dtype=torch.float64)
#Loss 25 = tensor(33.4970, dtype=torch.float64)
#Loss 26 = tensor(32.2919, dtype=torch.float64)
#Loss 27 = tensor(30.4696, dtype=torch.float64)
#Loss 28 = tensor(28.3404, dtype=torch.float64)
#Loss 29 = tensor(26.2762, dtype=torch.float64)
#Loss 30 = tensor(24.3320, dtype=torch.float64)
#Loss 31 = tensor(23.0128, dtype=torch

#Loss 11 = tensor(90.8764, dtype=torch.float64)
#Loss 12 = tensor(83.6942, dtype=torch.float64)
#Loss 13 = tensor(75.1360, dtype=torch.float64)
#Loss 14 = tensor(66.4467, dtype=torch.float64)
#Loss 15 = tensor(57.7199, dtype=torch.float64)
#Loss 16 = tensor(47.5285, dtype=torch.float64)
#Loss 17 = tensor(40.2411, dtype=torch.float64)
#Loss 18 = tensor(35.2290, dtype=torch.float64)
#Loss 19 = tensor(32.3965, dtype=torch.float64)
#Loss 20 = tensor(30.0683, dtype=torch.float64)
#Loss 21 = tensor(28.1232, dtype=torch.float64)
#Loss 22 = tensor(26.5938, dtype=torch.float64)
#Loss 23 = tensor(25.3942, dtype=torch.float64)
#Loss 24 = tensor(23.7493, dtype=torch.float64)
#Loss 25 = tensor(22.5503, dtype=torch.float64)
#Loss 26 = tensor(21.8974, dtype=torch.float64)
#Loss 27 = tensor(21.3508, dtype=torch.float64)
#Loss 28 = tensor(20.6789, dtype=torch.float64)
#Loss 29 = tensor(20.2412, dtype=torch.float64)
#Loss 30 = tensor(19.8779, dtype=torch.float64)
#Loss 31 = tensor(19.5229, dtype=torch.f